In [32]:
using FFTW
using GLMakie

Given a time-horizon $T > 0$, consider a real-valued signal $f:[0, T] \rightarrow \mathbb{R}$.

In [33]:
time_horizon = 10.0
t = range(0.0, time_horizon, 300)

frequencies = [1.0, 2.0, 5.0] / time_horizon
signal = sum(frequencies) do ν
    cos.(2π .* ν .* t)
end

fig, _ = lines(t, signal, color=:black, label="signal", axis=(; xlabel="time", ylabel="signal"), figure=(; size=(500, 500)))
display(fig)

GLMakie.Screen(...)

Once converted to digital, this signal usually consists in $N\in \mathbb{N}^*$ samples uniformly separated by $\delta_t>0$ units of time:
$$T = N\delta_t.$$
Denote $F \in \mathbb{R}^N$ the sampled signal: 
$$F_k = f(k\delta_t), \qquad 0 \leq k \leq N-1$$

In [34]:
sample_indices = 1:10:length(t)
samples = t[sample_indices]

signal_sampled = signal[sample_indices]
n_samples = length(signal_sampled)

fig, ax, _ = lines(t, signal, color=:black, label="signal", axis=(; xlabel="time", title="signal: $(n_samples) samples"), figure=(; size=(500, 500)))
scatter!(ax, samples, signal_sampled, color=:red, label="samples")
axislegend(ax)
display(fig)

GLMakie.Screen(...)

Given the time-horizon $T$, a natural choice of oscillating function are cosines and sines with a fundamental frequency $\kappa_1 = 1/T$, that is $g_1(t) = \exp(i 2\pi \kappa_1 t).$
Once evaluated at the instants $k\delta_t$ (i.e. sampled) yields the vector $G_1\in \mathbb{C}^N$ whose components are
$$G_{1,k} = \exp\left(i 2\pi \frac k N\right), \qquad 0 \leq k \leq N-1$$
Similarly, the mean component $g_0$ and the $N-2$ subsequent harmonics $g_n(t) = g_1(nt)$ lead to vectors $G_{n,k} = \exp\left(i 2\pi nk/N \right)$.

In [35]:
G(t, n) = exp(im * 2π * n * t/time_horizon)
G1, G2 = map(1:2) do n
    G.(t, n)
end

fig, ax11, _ = lines(t, real(G1), color=:red, label=L"\cos(2\pi\kappa_1)", axis=(; xlabel="time", title="fundamental oscillations"), figure=(; size=(500, 1000)))
lines!(ax11, t, real(G2), color=:blue, linestyle=:dash, label=L"\cos(2\pi\kappa_2)")
ax21, _ = lines(fig[2, 1], t, imag(G1), color=:red, label=L"\sin(2\pi\kappa_1)", axis=(; xlabel="time"))
lines!(ax21, t, imag(G2), color=:blue, linestyle=:dash, label=L"\sin(2\pi\kappa_2)")
axislegend(ax11)
axislegend(ax21)
display(fig)

GLMakie.Screen(...)

The discrete Fourier transform (DFT) is the mapping $\mathbb{C}^N \ni F \mapsto \hat F \in \mathbb{C}^N$ where 
$$\hat F_n = G_n^* F = \sum_{k=0}^{N-1} F_k \exp\left(-i 2\pi \frac k N\right),$$
or matrix-wise 
$$\hat F = G^* F, \qquad G = \begin{pmatrix} G_0 & G_1 & \dots & G_{N-1}\end{pmatrix} \in M_N(\mathbb{C}).$$

The $n^{th}$-coefficient of the DFT answers the question: quantify how much oscillations of frequency $\kappa_n = n/T$ make-up the *sampled* signal $F$ (hopefully the signal $f$ itself)?
The FFT is an algorithm which computes the DFT efficiently.

In [36]:
frequency_gap = 1/time_horizon # sampling rate
fourier_frequencies = frequency_gap * 0:n_samples-1
signal_dft = fft(signal_sampled)

30-element Vector{ComplexF64}:
   0.2918881412148732 + 0.0im
   15.252963942132652 + 0.15993989481099016im
   15.097671459924516 + 0.3166642264872082im
  0.05371127908835914 + 0.0023134424118949684im
  0.22536715577155997 + 0.010290852470711882im
   14.965446060619211 + 0.7853807400423544im
   -0.256272718036095 - 0.014805257362804405im
 -0.12575637322413868 - 0.007656694601836378im
 -0.08279196960970839 - 0.005092100137399525im
 -0.06186647652183071 - 0.0036789207144692386im
                      ⋮
  -0.0618664765218307 + 0.0036789207144692386im
 -0.08279196960970792 + 0.005092100137399108im
 -0.12575637322413913 + 0.007656694601835559im
   -0.256272718036095 + 0.014805257362804405im
   14.965446060619211 - 0.7853807400423543im
   0.2253671557715601 - 0.010290852470711074im
  0.05371127908835914 - 0.0023134424118949693im
   15.097671459924516 - 0.31666422648720804im
   15.252963942132652 - 0.15993989481099127im

In [69]:
fig, ax, _ = lines(t, signal, color=:black, label="signal", axis=(; xlabel="time", title="signal: $(n_samples) samples"), figure=(; size=(1500, 500)))
scatter!(ax, samples, signal_sampled, color=:red, label="samples")
axislegend(ax)

ax, _ = stem(fig[1, 2], fourier_frequencies, real(signal_dft), color=:red, stemcolor=:red, label=L"\mathrm{real}(\hat{F}_k)", axis=(; xlabel="frequency (Hz)", title="signal DFT"))
stem!(ax, fourier_frequencies, imag(signal_dft), color=:blue, stemcolor=:blue, label=L"\mathrm{imag}(\hat{F}_k)")
axislegend(ax)

energy = sum(abs2, signal_dft)
ax, _ = stem(fig[1, 3], fourier_frequencies, abs2.(signal_dft)./energy, color=:purple, stemcolor=:purple, label=L"|\hat{F}_k|^2/|\hat{F}|^2", axis=(; xlabel="frequency (Hz)", title="relative energy"))
axislegend(ax)
display(fig)

GLMakie.Screen(...)

Up to rescaling, the DFT matrix $G$ is an isometry 
$$GG^* = G^*G = N I_N.$$
This implies Parseval's relation which states that a sampled signal's energy equals the energy of its Fourier components
$$\sum_{k=1}^{N-1} |F_k|^2 = \frac{1}{N} \sum_{k=0}^{N-1}|\hat F_k|^2$$

In [38]:
signal_energy = sum(abs2, signal_sampled)
mean_fourier_energy = sum(abs2, signal_dft)/n_samples
@show signal_energy
@show mean_fourier_energy
nothing

signal_energy = 45.69982004305361
mean_fourier_energy = 45.69982004305361


Notice the horizontal symmetry of the modulus of Fourier components for the real-valued sampled-signal $F$: 
$$|F_n| = |F_{N - n}|, \qquad 1 \leq n \leq \operatorname{floor}(N/2)$$

Similarly, the real and imaginary parts satisfy
$$\Re(\hat F_n) = \Re(\hat F_{N - n}), \qquad \Im(\hat F_n) = -\Im(\hat F_{N - n})$$
because cos (resp. sin) is an even (resp. odd) function while the frequencies with indices $n \geq \operatorname{floor}(N/2)$ appear due to discretization as if they were negative frequencies $n-\operatorname{floor}(N/2)$.

Applying the DFT four times yields a sampled signal $F$ yields a scaled-up version of itself
$$(G^*)^4F = N^2 F \Longrightarrow (G^*)^{-1} = \frac{1}{N^2}(G^*)^3.$$
The ‘'isometry'' property also yields 
$$(G^*)^{-1} = \frac{1}{N}G.$$

In [39]:
signal_dft_fourtimes = real(fft(fft(fft(signal_dft)))) / n_samples^2
signal_inv_dft = real(ifft(signal_dft))

fig, ax, _ = lines(t, signal, color=:black, label="signal", axis=(; xlabel="time", title="signal: $(n_samples) samples"), figure=(; size=(1000, 500)))
scatter!(ax, samples, signal_dft_fourtimes, color=:red, label="DFT four times (scaled)")
stem!(ax, samples, signal_inv_dft, color=:blue, markersize=0, stemcolor=:blue, label="inverse DFT)")
axislegend(ax)
display(fig)

GLMakie.Screen(...)

Consider $\tau \in (0, 1]$ a compression *threshold*. It is possible to compress the original sampled signal by thresholding the $M_\tau\leq N$ Fourier components which contribute the least to the total energy. 

Write 
$$w_k = \frac{|\hat{F}_k|^2}{\sum_{j=0}^{N-1} |\hat{F}_j|^2}.$$
Consider then $\hat{F}^{(\tau)}\in \mathbb{C}^N$ such that 
$$\hat{F}^{(\tau)}_k = \hat{F}_k \mathbb{1}_{(\tau, 1]}(w_k),$$
i.e. which equals the Fourier components of $F$ provided it contributes to at least $\tau$ percent of the total energy of the sampled signal.

We refer by *compression rate* to the relative number of components discard in the compressed signal:
$$\mathrm{CR}_\tau = \frac{N - M_\tau}{N}$$
The higuer the compression rate, the less components we keep.

In [70]:
threshold = 0.15

weights = abs2.(signal_dft)./sum(abs2, signal_dft)
signal_dft_compressed = [(w > threshold) ? c : zero(c) for (c, w) in zip(signal_dft, weights)]

signal_compressed = real(ifft(signal_dft_compressed))
compression_error = signal_compressed - signal_sampled
keep = sum(weights) do w 
    (w > threshold) ? 1 : 0
end
compression_rate = (n_samples - keep)/n_samples


percent_compression_rate = round(compression_rate*100, digits=2)
percent_threshold = round(threshold*100, digits=2)

fig, ax, _ = lines(t, signal, color=:black, label="signal", 
    axis=(; xlabel="time", title="compression rate: $(percent_compression_rate)%, threshold $(percent_threshold)%, "),
    figure=(; size=(500, 1000))
)
stem!(ax, samples, signal_compressed, color=:blue, stemcolor=:blue, label="compressed")
stem(fig[2, 1], samples, compression_error, color=:red, stemcolor=:red, axis=(; xlabel="time", title="compression error"))
axislegend(ax)
display(fig)

GLMakie.Screen(...)